In [ ]:
 #!pip install scikit-lego -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 216.1/216.1 kB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.1/70.1 kB 3.9 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import numpy as np
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import train_test_split
from sklego.meta import ZeroInflatedRegressor
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import mean_absolute_error
%matplotlib inline

In [ ]:
#Загружаем данные, содержащие информацию по всем товарам во всех магазинах за все периоды времени (тренировочные + для которых необходимо предсказать спрос)
all_data = pd.read_csv('train.csv', delimiter=',')
all_data

,period_dt,NUM_CONSULTANT,PRICE_AFTER_DISC,PRICE_REGULAR,PRODUCT_LVL_RK6,PROMO1_FLAG,PROMO2_FLAG,STORE_LOCATION_LVL_RK2,STORE_LOCATION_LVL_RK3,STORE_LOCATION_LVL_RK4,demand,location_id,product_id,id
0,2017-01-02,0.0,1536.75,2049.0,73057,1.0,0.0,10,203,203,0.004237,425,19523,0
1,2017-01-09,0.0,1536.75,2049.0,73057,1.0,0.0,10,203,203,0.000000,425,19523,1
2,2017-01-16,0.0,1536.75,2049.0,73057,1.0,0.0,10,203,203,0.000000,425,19523,2
3,2017-01-23,0.0,1536.75,2049.0,73057,1.0,0.0,10,203,203,0.000000,425,19523,3
4,2017-01-30,0.0,1536.75,2049.0,73057,1.0,0.0,10,203,203,0.000000,425,19523,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
209527,2018-04-02,0.0,724.50,1449.0,9995,1.0,0.0,10,189,189,0.000000,1162,74708,209527
209528,2018-04-09,0.0,724.50,1449.0,9995,1.0,0.0,10,189,189,0.000000,1162,74708,209528
209529,2018-04-16,0.0,724.50,1449.0,9995,1.0,0.0,10,189,189,0.000000,1162,74708,209529
209530,2018-04-23,0.0,724.50,1449.0,9995,1.0,0.0,10,189,189,0.000000,1162,74708,209530


In [ ]:
all_data.columns.tolist()

['period_dt',
 'NUM_CONSULTANT',
 'PRICE_AFTER_DISC',
 'PRICE_REGULAR',
 'PRODUCT_LVL_RK6',
 'PROMO1_FLAG',
 'PROMO2_FLAG',
 'STORE_LOCATION_LVL_RK2',
 'STORE_LOCATION_LVL_RK3',
 'STORE_LOCATION_LVL_RK4',
 'demand',
 'location_id',
 'product_id',
 'id']

In [ ]:
#Посмотрим на уникальные значения категориальных объясняющих переменных
#и удалим те, у которых уникальное значение (не пустое) только одно, т.к. они не дают никакой пользы при обучении модели
print(all_data['PROMO1_FLAG'].unique())
print(all_data['PROMO2_FLAG'].unique())
print(all_data['NUM_CONSULTANT'].unique())
print(all_data['PRODUCT_LVL_RK6'].unique())
print(all_data['STORE_LOCATION_LVL_RK2'].unique()) # УДАЛИТЬ
print(all_data['STORE_LOCATION_LVL_RK3'].unique())
print(all_data['STORE_LOCATION_LVL_RK4'].unique())

[ 1.  0.  2. nan]
[ 0. nan  1.]
[ 0. nan  5.  4.  7.  6.  3.  2.  1.]
[ 73057   6206   9993 113186   9999 113196   9995]
[10]
[203  54 219 189  36 101  28  26  39 258 125  32  25]
[203  54 219 189  36 101  28  26  39 258 125  32  25]


In [ ]:
del all_data['STORE_LOCATION_LVL_RK2'] #не используем данную переменную, т.к. 1 уникальное значение

In [ ]:
#Поработаем с пропущенными значениями
#Пропуски в категориальных переменных заполним соответствующими модами:
all_data['PROMO1_FLAG'] = all_data['PROMO1_FLAG'].fillna(all_data['PROMO1_FLAG'].mode()[0])
all_data['PROMO2_FLAG'] = all_data['PROMO2_FLAG'].fillna(all_data['PROMO2_FLAG'].mode()[0])
all_data['PRICE_AFTER_DISC'] = all_data['PRICE_AFTER_DISC'].fillna(all_data['PRICE_AFTER_DISC'].mode()[0])
all_data['NUM_CONSULTANT'] = all_data['NUM_CONSULTANT'].fillna(all_data['NUM_CONSULTANT'].mode()[0])
all_data

,period_dt,NUM_CONSULTANT,PRICE_AFTER_DISC,PRICE_REGULAR,PRODUCT_LVL_RK6,PROMO1_FLAG,PROMO2_FLAG,STORE_LOCATION_LVL_RK3,STORE_LOCATION_LVL_RK4,demand,location_id,product_id,id
0,2017-01-02,0.0,1536.75,2049.0,73057,1.0,0.0,203,203,0.004237,425,19523,0
1,2017-01-09,0.0,1536.75,2049.0,73057,1.0,0.0,203,203,0.000000,425,19523,1
2,2017-01-16,0.0,1536.75,2049.0,73057,1.0,0.0,203,203,0.000000,425,19523,2
3,2017-01-23,0.0,1536.75,2049.0,73057,1.0,0.0,203,203,0.000000,425,19523,3
4,2017-01-30,0.0,1536.75,2049.0,73057,1.0,0.0,203,203,0.000000,425,19523,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...
209527,2018-04-02,0.0,724.50,1449.0,9995,1.0,0.0,189,189,0.000000,1162,74708,209527
209528,2018-04-09,0.0,724.50,1449.0,9995,1.0,0.0,189,189,0.000000,1162,74708,209528
209529,2018-04-16,0.0,724.50,1449.0,9995,1.0,0.0,189,189,0.000000,1162,74708,209529
209530,2018-04-23,0.0,724.50,1449.0,9995,1.0,0.0,189,189,0.000000,1162,74708,209530


In [ ]:
def has_valid_prices(group):
    return not (group['PRICE_REGULAR'].isna().all() and group['PRICE_AFTER_DISC'].isna().all())

all_data = all_data.groupby('location_id').filter(has_valid_prices)


In [ ]:
all_data['all_prices_na'] = all_data.groupby(['product_id', 'period_dt'])['PRICE_REGULAR'].transform(lambda x: x.isna().all())

# Фильтруем данные, чтобы найти те, где все значения цен NaN
missing_price_pairs = all_data[all_data['all_prices_na'] == True]

# Уникальные пары "товар+день", где цены неизвестны
unique_missing_price_pairs = missing_price_pairs[['product_id', 'period_dt']].drop_duplicates()

# Считаем количество таких пар
n = len(unique_missing_price_pairs)
listt = unique_missing_price_pairs.values.tolist()


In [ ]:
for pair in listt:
    product_id, date = pair
    all_data = all_data[(all_data['product_id'] != product_id) | (all_data['period_dt'] != date)]

In [ ]:
all_data

,period_dt,NUM_CONSULTANT,PRICE_AFTER_DISC,PRICE_REGULAR,PRODUCT_LVL_RK6,PROMO1_FLAG,PROMO2_FLAG,STORE_LOCATION_LVL_RK3,STORE_LOCATION_LVL_RK4,demand,location_id,product_id,id,all_prices_na
0,2017-01-02,0.0,1536.75,2049.0,73057,1.0,0.0,203,203,0.004237,425,19523,0,False
1,2017-01-09,0.0,1536.75,2049.0,73057,1.0,0.0,203,203,0.000000,425,19523,1,False
2,2017-01-16,0.0,1536.75,2049.0,73057,1.0,0.0,203,203,0.000000,425,19523,2,False
3,2017-01-23,0.0,1536.75,2049.0,73057,1.0,0.0,203,203,0.000000,425,19523,3,False
4,2017-01-30,0.0,1536.75,2049.0,73057,1.0,0.0,203,203,0.000000,425,19523,4,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
209527,2018-04-02,0.0,724.50,1449.0,9995,1.0,0.0,189,189,0.000000,1162,74708,209527,False
209528,2018-04-09,0.0,724.50,1449.0,9995,1.0,0.0,189,189,0.000000,1162,74708,209528,False
209529,2018-04-16,0.0,724.50,1449.0,9995,1.0,0.0,189,189,0.000000,1162,74708,209529,False
209530,2018-04-23,0.0,724.50,1449.0,9995,1.0,0.0,189,189,0.000000,1162,74708,209530,False


In [ ]:
a = 0
for i in all_data['product_id'].unique():
  for d in all_data[all_data['product_id'] == i]['period_dt'].unique():
    a += 1
a

18736

In [ ]:
median_prices = all_data.groupby(['product_id', 'period_dt'])['PRICE_REGULAR'].transform('median')

# Заполняем пропущенные значения в PRICE_REGULAR медианными значениями
all_data['PRICE_REGULAR'] = all_data['PRICE_REGULAR'].fillna(median_prices)


In [ ]:
median_discount_prices = all_data.groupby(['product_id', 'period_dt'])['PRICE_AFTER_DISC'].transform('median')

# Заполняем пропущенные значения в PRICE_AFTER_DISC медианными значениями
all_data['PRICE_AFTER_DISC'] = all_data['PRICE_AFTER_DISC'].fillna(median_discount_prices)

In [ ]:
#Проверка:
all_data['PRICE_REGULAR'].isnull().any()

False

In [ ]:
all_data['PRICE_AFTER_DISC'].isnull().any()

False

In [ ]:
all_data['period_dt'] = pd.to_datetime(all_data['period_dt'], dayfirst=False)
all_data["ind_of_year"] = all_data.period_dt.dt.year
all_data["ind_of_month"] = all_data.period_dt.dt.month
all_data["ind_of_day"] = all_data.period_dt.dt.day

all_data.head()

,period_dt,NUM_CONSULTANT,PRICE_AFTER_DISC,PRICE_REGULAR,PRODUCT_LVL_RK6,PROMO1_FLAG,PROMO2_FLAG,STORE_LOCATION_LVL_RK3,STORE_LOCATION_LVL_RK4,demand,location_id,product_id,id,all_prices_na,ind_of_year,ind_of_month,ind_of_day
0,2017-01-02,0.0,1536.75,2049.0,73057,1.0,0.0,203,203,0.004237,425,19523,0,False,2017,1,2
1,2017-01-09,0.0,1536.75,2049.0,73057,1.0,0.0,203,203,0.000000,425,19523,1,False,2017,1,9
2,2017-01-16,0.0,1536.75,2049.0,73057,1.0,0.0,203,203,0.000000,425,19523,2,False,2017,1,16
3,2017-01-23,0.0,1536.75,2049.0,73057,1.0,0.0,203,203,0.000000,425,19523,3,False,2017,1,23
4,2017-01-30,0.0,1536.75,2049.0,73057,1.0,0.0,203,203,0.000000,425,19523,4,False,2017,1,30


In [ ]:
#и теперь удалим переменную period_start_dt за ненадобностью
del all_data['period_dt']
del all_data['all_prices_na']
all_data

,NUM_CONSULTANT,PRICE_AFTER_DISC,PRICE_REGULAR,PRODUCT_LVL_RK6,PROMO1_FLAG,PROMO2_FLAG,STORE_LOCATION_LVL_RK3,STORE_LOCATION_LVL_RK4,demand,location_id,product_id,id,ind_of_year,ind_of_month,ind_of_day
0,0.0,1536.75,2049.0,73057,1.0,0.0,203,203,0.004237,425,19523,0,2017,1,2
1,0.0,1536.75,2049.0,73057,1.0,0.0,203,203,0.000000,425,19523,1,2017,1,9
2,0.0,1536.75,2049.0,73057,1.0,0.0,203,203,0.000000,425,19523,2,2017,1,16
3,0.0,1536.75,2049.0,73057,1.0,0.0,203,203,0.000000,425,19523,3,2017,1,23
4,0.0,1536.75,2049.0,73057,1.0,0.0,203,203,0.000000,425,19523,4,2017,1,30
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
209527,0.0,724.50,1449.0,9995,1.0,0.0,189,189,0.000000,1162,74708,209527,2018,4,2
209528,0.0,724.50,1449.0,9995,1.0,0.0,189,189,0.000000,1162,74708,209528,2018,4,9
209529,0.0,724.50,1449.0,9995,1.0,0.0,189,189,0.000000,1162,74708,209529,2018,4,16
209530,0.0,724.50,1449.0,9995,1.0,0.0,189,189,0.000000,1162,74708,209530,2018,4,23


In [ ]:
#Подготовим данные для обучения модели
#Поделим данные на трейн и тест
data_train = all_data[all_data['demand'].isnull() == False]
data_train

,NUM_CONSULTANT,PRICE_AFTER_DISC,PRICE_REGULAR,PRODUCT_LVL_RK6,PROMO1_FLAG,PROMO2_FLAG,STORE_LOCATION_LVL_RK3,STORE_LOCATION_LVL_RK4,demand,location_id,product_id,id,ind_of_year,ind_of_month,ind_of_day
0,0.0,1536.75,2049.0,73057,1.0,0.0,203,203,0.004237,425,19523,0,2017,1,2
1,0.0,1536.75,2049.0,73057,1.0,0.0,203,203,0.000000,425,19523,1,2017,1,9
2,0.0,1536.75,2049.0,73057,1.0,0.0,203,203,0.000000,425,19523,2,2017,1,16
3,0.0,1536.75,2049.0,73057,1.0,0.0,203,203,0.000000,425,19523,3,2017,1,23
4,0.0,1536.75,2049.0,73057,1.0,0.0,203,203,0.000000,425,19523,4,2017,1,30
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
209527,0.0,724.50,1449.0,9995,1.0,0.0,189,189,0.000000,1162,74708,209527,2018,4,2
209528,0.0,724.50,1449.0,9995,1.0,0.0,189,189,0.000000,1162,74708,209528,2018,4,9
209529,0.0,724.50,1449.0,9995,1.0,0.0,189,189,0.000000,1162,74708,209529,2018,4,16
209530,0.0,724.50,1449.0,9995,1.0,0.0,189,189,0.000000,1162,74708,209530,2018,4,23


In [ ]:
data_test = all_data[all_data['demand'].isnull()]
#Сразу переименуем столбец "demand" в тестовой выборке на "predicted"
data_test.rename(columns={'demand': 'predicted'}, inplace=True)
data_test

<ipython-input-26-cc8614a3c1da>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_test.rename(columns={'demand': 'predicted'}, inplace=True)


,NUM_CONSULTANT,PRICE_AFTER_DISC,PRICE_REGULAR,PRODUCT_LVL_RK6,PROMO1_FLAG,PROMO2_FLAG,STORE_LOCATION_LVL_RK3,STORE_LOCATION_LVL_RK4,predicted,location_id,product_id,id,ind_of_year,ind_of_month,ind_of_day
314,0.0,1049.50,2099.0,73057,1.0,0.0,189,189,NaN,1162,19523,314,2019,12,9
315,0.0,1049.50,2099.0,73057,1.0,0.0,189,189,NaN,1162,19523,315,2019,12,16
531,0.0,629.70,2099.0,73057,1.0,0.0,54,54,NaN,1328,19523,531,2019,12,9
532,0.0,629.70,2099.0,73057,1.0,0.0,54,54,NaN,1328,19523,532,2019,12,16
533,0.0,629.70,2099.0,73057,1.0,0.0,54,54,NaN,1328,19523,533,2019,12,23
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
209235,5.0,1499.00,1499.0,9995,1.0,0.0,203,203,NaN,1034,74708,209235,2019,12,16
209236,5.0,1499.00,1499.0,9995,1.0,0.0,203,203,NaN,1034,74708,209236,2019,12,23
209237,5.0,1124.25,1499.0,9995,1.0,0.0,203,203,NaN,1034,74708,209237,2019,12,30
209468,0.0,724.00,1499.0,9995,1.0,0.0,39,39,NaN,1143,74708,209468,2019,12,9


In [ ]:
X = data_train.drop(['id', 'demand'], axis=1)
y = data_train['demand']

In [ ]:
X.isna().sum()

NUM_CONSULTANT            0
PRICE_AFTER_DISC          0
PRICE_REGULAR             0
PRODUCT_LVL_RK6           0
PROMO1_FLAG               0
PROMO2_FLAG               0
STORE_LOCATION_LVL_RK3    0
STORE_LOCATION_LVL_RK4    0
location_id               0
product_id                0
ind_of_year               0
ind_of_month              0
ind_of_day                0
dtype: int64

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

In [ ]:
y_train.describe()

count    154949.000000
mean          0.107854
std           0.348366
min           0.000000
25%           0.000000
50%           0.000000
75%           0.000000
max           8.416667
Name: demand, dtype: float64

In [ ]:
# оптимальные параметры модели были подобраны вручную
# обучим модель и проверим ее качество

classifier = LogisticRegression()
xgb_regressor = GradientBoostingRegressor(random_state=1max_depth=11, n_estimators=420, learning_rate=0.2, random_state=1, min_samples_leaf=11, min_samples_split=2, loss='absolute_error')
zero_inflated = ZeroInflatedRegressor(classifier=classifier, regressor=xgb_regressor)

zero_inflated.fit(X_train, y_train)


# from sklearn.model_selection import GridSearchCV
# from sklearn.ensemble import GradientBoostingRegressor

# regressor = GradientBoostingRegressor(random_state=1)

# # Определяем сетку
# param_grid = {
#     'n_estimators': [100, 200, 300, 400, 420, 500],
#     'max_depth': [3, 5, 7, 9, 11],
#     'learning_rate': [0.01, 0.05, 0.1, 0.2],
#     'min_samples_split': [2, 5, 10],
#     'min_samples_leaf': [1, 5, 10, 11],
#     'loss': ['squared_error', 'absolute_error', 'huber', 'quantile']
# }

# grid_search = GridSearchCV(estimator=regressor, param_grid=param_grid, cv=5, n_jobs=-1, verbose=2)

# grid_search.fit(X_train, y_train)

# #лучшие параметры
# print("Best parameters found: ", grid_search.best_params_)
# regressor = GradientBoostingRegressor(
#     max_depth=11,
#     n_estimators=420,
#     learning_rate=0.2,
#     random_state=1,
#     min_samples_leaf=11,
#     min_samples_split=2,
#     loss='absolute_error'
# )
# regressor.fit(X_train, y_train)

ZeroInflatedRegressor(classifier=LogisticRegression(),
                      regressor=GradientBoostingRegressor(random_state=1))

In [ ]:
y_pred = zero_inflated.predict(X_test)
mean_absolute_error(y_test, y_pred)

0.10834771454017424

In [ ]:
#предскажем искомые значения спроса для нашей тестовой выборки (с декабря 2019)
X_test = data_test.drop(['id', 'predicted'], axis=1)
X_test

,NUM_CONSULTANT,PRICE_AFTER_DISC,PRICE_REGULAR,PRODUCT_LVL_RK6,PROMO1_FLAG,PROMO2_FLAG,STORE_LOCATION_LVL_RK3,STORE_LOCATION_LVL_RK4,location_id,product_id,ind_of_year,ind_of_month,ind_of_day
314,0.0,1049.50,2099.0,73057,1.0,0.0,189,189,1162,19523,2019,12,9
315,0.0,1049.50,2099.0,73057,1.0,0.0,189,189,1162,19523,2019,12,16
531,0.0,629.70,2099.0,73057,1.0,0.0,54,54,1328,19523,2019,12,9
532,0.0,629.70,2099.0,73057,1.0,0.0,54,54,1328,19523,2019,12,16
533,0.0,629.70,2099.0,73057,1.0,0.0,54,54,1328,19523,2019,12,23
...,...,...,...,...,...,...,...,...,...,...,...,...,...
209235,5.0,1499.00,1499.0,9995,1.0,0.0,203,203,1034,74708,2019,12,16
209236,5.0,1499.00,1499.0,9995,1.0,0.0,203,203,1034,74708,2019,12,23
209237,5.0,1124.25,1499.0,9995,1.0,0.0,203,203,1034,74708,2019,12,30
209468,0.0,724.00,1499.0,9995,1.0,0.0,39,39,1143,74708,2019,12,9


In [ ]:
#предскажем значения спроса для тестовой выборки
y_pred_res = zero_inflated.predict(X_test)
y_pred_res

array([0., 0., 0., ..., 0., 0., 0.])

In [ ]:
#создадим датафрейм y_results, в который поместим результаты
y_results = data_test[['id', 'predicted']]
y_results['predicted'] = y_pred_res
y_results

<ipython-input-35-3838a002919f>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  y_results['predicted'] = y_pred_res


,id,predicted
314,314,0.0
315,315,0.0
531,531,0.0
532,532,0.0
533,533,0.0
...,...,...
209235,209235,0.0
209236,209236,0.0
209237,209237,0.0
209468,209468,0.0


In [ ]:
# удалим отрицательыне прогнозы из выборки
y_results.loc[y_results['predicted'] < 0, ['predicted']]

,predicted


In [ ]:
#заменим все отрицательные предсказанные значения спроса на ноль
y_results.loc[y_results['predicted'] < 0, ['predicted']] = 0
y_results.loc[y_results['predicted'] < 0, ['predicted']]

,predicted


In [ ]:
y_results.rename(columns={'predicted': 'demand'}, inplace=True)
y_results

<ipython-input-38-98baa8cd6fdc>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  y_results.rename(columns={'predicted': 'demand'}, inplace=True)


,id,demand
314,314,0.0
315,315,0.0
531,531,0.0
532,532,0.0
533,533,0.0
...,...,...
209235,209235,0.0
209236,209236,0.0
209237,209237,0.0
209468,209468,0.0


In [ ]:
#Записываем полученный датафрейм в csv файл:
y_results.to_csv('my_submittion.csv',sep=',', encoding='utf-8', index=False)

In [ ]:
sample_submission = pd.read_csv('sample_submission.csv', delimiter=',')
sample_submission

,id,demand
0,46105,1.0
1,73296,1.0
2,117042,1.0
3,117043,1.0
4,118221,1.0
...,...,...
368,779205,1.0
369,779206,1.0
370,779208,1.0
371,779209,1.0


In [ ]:
final_results = sample_submission[['id']].merge(y_results, on='id', how='left')
final_results['demand'] = final_results['demand'].fillna(0)
final_results.describe()

,id,demand
count,373.000000,373.0
mean,594350.431635,0.0
std,164795.721099,0.0
min,46105.000000,0.0
25%,530875.000000,0.0
50%,603945.000000,0.0
75%,749400.000000,0.0
max,779210.000000,0.0


In [ ]:
final_results.to_csv('my_submittion_final.csv', index=False)